In [1]:
import torch, cv2
from pathlib import Path
from PIL import Image
from transformers import SiglipVisionModel, SiglipImageProcessor

ROOT_IMG  = Path("/home/s2behappy4/data/gyuhyeong/dataset/MMAD/MVTec-AD/hazelnut/test")
ROOT_SAVE = Path("/home/s2behappy4/data/gyuhyeong/code/siglip_global/hazelnut")
TYPES     = ["crack", "cut", "hole", "print"]

DEV, MODEL_ID = ("cuda" if torch.cuda.is_available() else "cpu",
                 "google/siglip-so400m-patch14-384")

proc    = SiglipImageProcessor.from_pretrained(MODEL_ID)
siglip  = SiglipVisionModel.from_pretrained(MODEL_ID).to(DEV).eval()

In [ ]:
for defect in TYPES:
    img_dir  = ROOT_IMG  / defect
    save_dir = ROOT_SAVE / defect
    save_dir.mkdir(parents=True, exist_ok=True)

    for img_path in img_dir.glob("*.png"):
        # 1) Image → SigLIP
        pix = proc(images=Image.open(img_path).convert("RGB"),
                   return_tensors="pt")["pixel_values"].to(DEV)

        # 2) MAP-head (global token)
        with torch.no_grad():
            g_tok = siglip(pixel_values=pix).pooler_output.squeeze(0).cpu()  

        # 3) save
        torch.save(
            {"img_name": img_path.name, "global_token": g_tok},
            save_dir / f"{img_path.stem}_global_token.pt"
        )
        print(f"[{defect}] saved: {img_path.stem}")